## YOLOv8 Coin Detection - Training Notebook

### Fix Dataset Structure

In [ ]:
import yaml
import os

DATASET_PATH = "resources/manual_annotated_COIN_dataset"
yaml_path = os.path.join(DATASET_PATH, "data.yaml")

with open(yaml_path, "r") as f:
    data = yaml.safe_load(f)

data = {
    "path": DATASET_PATH,
    "train": "train/images",
    "val": "valid/images",
    "test": "test/images",
    "names": {
        0: "10_peso",
        1: "1_peso",
        2: "20_peso",
        3: "5_peso"
    }
}

with open(yaml_path, "w") as f:
    yaml.safe_dump(data, f)

data



{'train': '../train/images',
 'val': '../valid/images',
 'test': '../test/images',
 'nc': 4,
 'names': ['10_peso', '1_peso', '20_peso', '5_peso'],
 'roboflow': {'workspace': 'philippine-coin-counter-csc173',
  'project': 'ph-coin-counter',
  'version': 2,
  'license': 'MIT',
  'url': 'https://universe.roboflow.com/philippine-coin-counter-csc173/ph-coin-counter/dataset/2'}}

### Sanity check labels

In [10]:
from collections import Counter
import glob
import os

DATASET_PATH = "resources/manual_annotated_COIN_dataset"

label_files = glob.glob(os.path.join(DATASET_PATH, "train", "labels", "*.txt"))

class_counts = Counter()

for lf in label_files:
    with open(lf) as f:
        for line in f:
            cls = int(line.split()[0])
            class_counts[cls] += 1

class_counts


Counter({1: 344, 3: 250, 2: 188, 0: 172})

### Load YOLOv8 Model (Transfer Learning) and Train the Model

In [ ]:
from ultralytics import YOLO

# Uses YOLOv8 nano (fast + sufficient for coins)
model = YOLO("yolov8n.pt")

model.train(
    data=f"{DATASET_PATH}/data.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    patience=10,
    project="coin_yolo_training",
    name="yolov8n_coins"
)


### Key settings explained

| Parameter   | Meaning                       |
| ----------- | ----------------------------- |
| epochs=50   | enough for convergence        |
| imgsz=640   | matches dataset preprocessing |
| batch=16    | safe for Colab GPU            |
| patience=10 | early stopping                |


### Evaluate Model on Test Set

In [ ]:
from ultralytics import YOLO

model = YOLO("coin_yolo_training/yolov8n_coins/weights/best.pt")

test_results = model.val(
    data=f"{DATASET_PATH}/data.yaml",
    split="test",
    imgsz=640
)

print(test_results)

